Axioms for the theory of <a class="ProveItLink" href="theory.ipynb">proveit.physics.quantum.QPE</a>
========

In [ ]:
import proveit
# Prepare this notebook for defining the axioms of a theory:
%axioms_notebook # Keep this at the top following 'import proveit'.
from proveit import Literal, ExprArray, ExprRange, ExprTuple
from proveit import a, b, c, d, e, eps, k, l, m, n, r, s, t, U
from proveit.logic import Equals, Forall, NotEquals, InSet, Set
from proveit.numbers import (zero, one, two, three, i, pi, Div, Exp, Abs,
                             Integer, Interval, IntervalCO, Natural, NaturalPos, Neg)
from proveit.numbers import (zero, one, two, three, i, pi, Div, Exp,
                             Integer, Interval, IntervalCO, IntervalOC, Natural, NaturalPos, Neg)
from proveit.numbers import (Add, exp, Ceil, Floor, frac, greater_eq, LessEq, Mod, ModAbs, 
                             Mult, Prod, subtract, Sum, Log, sqrt)
from proveit.linear_algebra import (
    VecAdd, MatrixMult, ScalarMult, TensorExp, TensorProd, Norm)
from proveit.physics.quantum import (
    Meas, NumBra, NumKet, RegisterU, Qmult,
    QubitRegisterSpace, ket0, ket1)
#from proveit.physics.quantum.circuit import (Circuit, Input, Output, Gate, MultiQubitGate, 
#                                             IdentityOp, MultiWire, CircuitEquiv)
from proveit.physics.quantum.circuits import QcircuitEquiv
from proveit.physics.quantum.QPE import (
    _eps, _alpha_l, _b, _delta, _n, _s, _t, _phase, _phase_est, _psi_t_ket, _psi__t_ket, _Psi_ket,
    _two_pow_t, two_pow_t, _ket_u, _U, _U_pow_two_pow_k,
    #p_0, p_k, p_r, p_prime, p_prime_0, p_prime_k, p_prime_r, psi_prime, psi_prime_1, psi_prime_t,
    _QPE_U_t, _QPE1_U_t, _QPE_U_t_circuit, QPE1_U_t_circuit, 
    _psi_t_circuit, _Psi_circuit, _phase_est_circuit, 
    _Omega, _sample_space, _e_domain, _success_prob_e, _fail_prob_e)
from proveit.physics.quantum.QPE.phase_est_ops import ModAdd, Pfail, Psuccess, SubIndexed
from proveit.physics.quantum.QFT import InverseFourierTransform
from proveit.statistics import Prob

In [ ]:
%begin axioms

### Quantum circuit definitions

In [ ]:
QPE1_def = Forall(
    (s, t), Forall(
        U, QcircuitEquiv(_QPE1_U_t.literals_as_variables(_U, _t, _s), 
                         QPE1_U_t_circuit),
        domain=RegisterU(s)),
    domain=NaturalPos)

In [ ]:
QPE_def = Forall(
    (s, t), Forall(
        U, QcircuitEquiv(_QPE_U_t.literals_as_variables(_U, _t, _s), 
                         _QPE_U_t_circuit.literals_as_variables(_U, _t, _s)),
        domain=RegisterU(s)),
    domain=NaturalPos)

## Local axioms for convenience (for internal use only)

Let $U$ be a unitary operator that acts on $s$ qubits, with $\lvert u\rangle$ as an eigenstate of $U$ with eigenvalue $e^{2 \pi i \varphi}$:

In [ ]:
_s_in_nat_pos = InSet(_s, NaturalPos)

In [ ]:
_unitary_U = InSet(_U, RegisterU(_s))

In [ ]:
_u_ket_register = InSet(_ket_u, QubitRegisterSpace(_s))

In [ ]:
_normalized_ket_u = Equals(Norm(_ket_u), one)

In [ ]:
_phase_in_interval = InSet(_phase, IntervalCO(zero, one))

In [ ]:
_eigen_uu = Equals(MatrixMult(_U, _ket_u), 
                   ScalarMult(exp(Mult(two, pi, i, _phase)), _ket_u))

In [ ]:
# This local axiom is only used for the ideal case (when the phase can be represented exactly with t bits)
_ideal_phase_cond = InSet(Mult(_two_pow_t, _phase), 
                          Interval(zero, subtract(_two_pow_t, one)))

In [ ]:
# t (represented by the Literal t_)
# denotes the number of Qbits in the input register
_t_in_natural_pos = InSet(_t, NaturalPos)

In [ ]:
_psi_t_def = Forall(
    t, Equals(
        _psi_t_ket,
        TensorProd(ExprRange(r, 
                             ScalarMult(frac(one, sqrt(two)),
                                        VecAdd(ket0, 
                                               ScalarMult(exp(Mult(two, pi, i, Exp(two, r), _phase)),
                                                          ket1))), 
                             subtract(t, one), zero, order='decreasing'))),
    domain=NaturalPos)

In [ ]:
_Psi_def = Equals(_Psi_ket, Qmult(InverseFourierTransform(_t),
                                  _psi__t_ket))

Let $\lvert \Psi \rangle$ be the outcome of the $t$-qubit register of ${\rm QPE}(U, t)$ acting on $\lvert u \rangle$, $m$ be a random variable representing the measurement of Psi with the register interpreted as an integer (via binary representation), and $\varphi_m = 2 \pi m/2^t$ be the random variable phase outcome of ${\rm QPE}(U, t)$:

Let $b$ be the value for $m$ that gives the closest $\varphi_m$ to $\varphi$ without exceeding it, and let $\delta$ be this difference:

In [ ]:
_best_def = Equals(_b, Floor(Mult(_phase, _two_pow_t)))

In [ ]:
_delta_def = Equals(_delta, subtract(_phase, frac(_b, _two_pow_t)))

In [ ]:
_sample_space_def = Equals(_Omega, _sample_space)

The probability of success is defined as $\theta_m$ being within some epsilon of $\theta$: 

In [ ]:
_success_def = Forall(
        e,
        Equals(Psuccess(e),
               _success_prob_e),
        domain=_e_domain)

In [ ]:
_fail_def = Forall(
    e,
    Equals(Pfail(e), _fail_prob_e),
    domain=_e_domain)

Let $\alpha_l$ be the amplitude of $\lvert \Psi \rangle$ for the $b \oplus l$ state, where $\oplus$ is defined as addition modulo $2^t$:

In [ ]:
_mod_add_def = Forall(
    (a, b),
    Equals(ModAdd(a, b), Mod(Add(a, b), _two_pow_t)),
    domain=Integer)

In [ ]:
_alpha_def = Forall(
    l,
    Equals(_alpha_l,
           Qmult(NumBra(ModAdd(_b, l), _t),
                 _Psi_ket)),
    domain=Integer)

In [ ]:
# n (represented by the Literal _n)
# denotes the desired bits of precision (and thus n ≤ t)
_n_in_natural_pos = InSet(_n, NaturalPos)

In [ ]:
# n (represented by the Literal _n)
# denotes the desired bits of precision (and thus n ≤ t)
# n ≥ 2 helps with some details in some of the final steps of
# the overall general theorem, but the assumption should be revisited later
_n_ge_two = greater_eq(_n, two)

In [ ]:
_eps_in_interval = InSet(_eps, IntervalOC(zero, one))

In [ ]:
_t_req = greater_eq(
    _t, Add(_n, Ceil(Log(two, Add(two, frac(one, Mult(two, _eps)))))))

In [ ]:
%end axioms